In [1]:
%matplotlib inline

import os
from glob import glob

import yaml
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib import ticker
import seaborn
import numpy
import pandas as pd
import networkx
import dinopy
import pybedtools

from sh import bwa, samtools

from phasm.io import gfa
from phasm.assembly_graph import AssemblyGraph
from phasm.bubbles import find_superbubbles

BASE_DIR = os.path.realpath(os.path.join(os.getcwd(), '..'))
ALIGNED_CONTIGS_FILE = os.path.join(
    BASE_DIR, "assemblies/{assembly}/05_analysis/aligned_contigs.bam")

with open(os.path.join(BASE_DIR, "config.yml")) as f:
    config = yaml.load(f)
    
#seaborn.set_context('paper')
seaborn.set_style('whitegrid')

## Align contigs to reference

In [6]:
for assembly, asm_config in config['assemblies'].items():
    parts = assembly.split('-')
    ploidy = int(parts[0].replace("ploidy", ""))
    coverage = int(parts[1].replace("x", ""))
    
    if coverage != 60:
        continue
    
    asm_dir = os.path.join(BASE_DIR, "assemblies", assembly)
    ref_fasta = os.path.join(BASE_DIR, asm_config['reference'])
    bam_file = ALIGNED_CONTIGS_FILE.format(assembly=assembly)
    contigs_fasta = os.path.join(asm_dir, "04_phase", assembly + ".fasta")
    
    os.makedirs(os.path.dirname(bam_file), exist_ok=True)
    
    if not os.path.isfile(ref_fasta + '.bwt'):
        print("Create reference index...")
        # No BWA index available, create one
        bwa.index(ref_fasta)
    
    if not os.path.isfile(bam_file) and os.path.isfile(contigs_fasta):
        print("Creating bam file...")
        samtools.sort(samtools.view(bwa.mem(ref_fasta, contigs_fasta), '-uS', '-'), '-o', bam_file)
        
    if not os.path.isfile(bam_file + '.bai') and os.path.isfile(bam_file):
        samtools.index(bam_file)
    
    print()

Creating bam file...

Creating bam file...

Creating bam file...

Creating bam file...




## Assembly metrics

In [5]:
data = []
for assembly, asm_config in config['assemblies'].items():
    parts = assembly.split('-')
    ploidy = int(parts[0].replace("ploidy", ""))
    coverage = int(parts[1].replace("x", ""))
    
    asm_dir = os.path.join(BASE_DIR, "assemblies", assembly)
    ref_fasta = os.path.join(BASE_DIR, asm_config['reference'])
    bam_file = ALIGNED_CONTIGS_FILE.format(assembly=assembly)
    contigs_fasta = os.path.join(asm_dir, "04_phase", assembly + ".fasta")
    
    if coverage != 60:
        continue
    
    if not os.path.isfile(contigs_fasta):
        continue
    
    ref_reader = dinopy.FastaReader(ref_fasta)
    ref_length = sum(e.length for e in ref_reader.entries())
    
    theo_ng50 = ref_length / 2
    ng50 = 0
    
    contig_reader = dinopy.FastaReader(contigs_fasta)
    total_contig_length = 0
    total_haploblock_length = 0
    contig_sequence_len = 0
    contig_lengths = []
    for contig in contig_reader.entries():
        total_contig_length += contig.length
        
        if theo_ng50 <= total_contig_length:
            ng50 = contig.length
            
        name = contig.name.decode('utf-8')
        if "haploblock" in name:
            total_haploblock_length += contig.length
        else:
            contig_sequence_len += contig.length
        
        contig_lengths.append(contig.length)
        
    data.append({
        'ploidy': ploidy,
        'coverage': coverage,
        'num_contigs': len(contig_lengths),
        'num_contigs_norm': len(contig_lengths) / ploidy,
        'ng50': ng50,
        'longest_contig': max(contig_lengths),
        'average_contig_len': sum(contig_lengths) / len(contig_lengths),
        'haploblock_fraction': total_haploblock_length / total_contig_length,
        'contig_fraction': contig_sequence_len / total_contig_length
    })
        
df = pd.DataFrame(data)
df 

,average_contig_len,contig_fraction,coverage,haploblock_fraction,longest_contig,ng50,num_contigs,num_contigs_norm,ploidy
0,128312.863636,0.181175,60,0.818825,282556,39952,22,11.000000,2
1,88329.217391,0.078573,60,0.921427,195203,195197,46,15.333333,3
2,130623.625000,0.000000,60,1.000000,328364,122790,48,12.000000,4
3,195952.229167,0.000000,60,1.000000,307809,209731,48,8.000000,6


## Percentage of reference genome covered

In [ ]:
def contig_filter(f, name):
    return f[0] == name

for assembly, asm_config in config['assemblies'].items():
    parts = assembly.split('-')
    ploidy = int(parts[0].replace("ploidy", ""))
    coverage = int(parts[1].replace("x", ""))
    
    asm_dir = os.path.join(BASE_DIR, "assemblies", assembly)
    ref_fasta = os.path.join(BASE_DIR, asm_config['reference'])
    bam_file = ALIGNED_CONTIGS_FILE.format(assembly=assembly)
    contigs_fasta = os.path.join(asm_dir, "04_phase", assembly + ".fasta")
    
    if not os.path.isfile(contigs_fasta):
        continue
        
    if coverage != 60:
        continue
    
    top_margin = 0.4  # inches
    bottom_margin = 0.6  # inches
    total_margin = top_margin + bottom_margin
    axis_height = 0.8
    hspace = 0.2
    all_axis_height = (ploidy*axis_height) + ((ploidy-1) * (axis_height*hspace))
    
    margin_fraction = total_margin / (all_axis_height + total_margin)
    top_margin_frac = (top_margin * margin_fraction) / total_margin
    bottom_margin_frac = (bottom_margin * margin_fraction) / total_margin
    
    fig_height = all_axis_height + total_margin
    fig, ax = plt.subplots(ploidy, sharex=True, figsize=(7, fig_height))
    bt = pybedtools.BedTool()
    fr = dinopy.FastaReader(ref_fasta)
    for i, homolog in enumerate(fr.entries()):
        name = homolog.name.decode('utf-8')
        chr_id = name.split()[0]
        
        cov = bt.genome_coverage(ibam=bam_file, bga=True, stream=True).filter(
            contig_filter, chr_id)
        
        coverage_data = numpy.zeros((homolog.length,))
        for region in cov:
            start = int(region[1])
            end = int(region[2])
            reg_coverage = int(region[3])
            
            coverage_data[start:end] = reg_coverage
        
        ax[i].plot(coverage_data)
        ax[i].set_xlim((0, 800000))
        ax[i].set_ylim((0, 15))
        ax[i].set_ylabel("Chr. {}".format(i+1))
        ax[i].yaxis.set_major_locator(ticker.MultipleLocator(5))
        
        ax[i].add_patch(
            patches.Rectangle((0, 0), homolog.length, 0.5, alpha=0.3)
        )
        
    fig.subplots_adjust(bottom=bottom_margin_frac,
                        top=(1-top_margin_frac), hspace=hspace)
    fig.text(0.5, (1-top_margin_frac)+(top_margin_frac/2), "Coverage by PHASM contigs (genome: ploidy {}, {}x read coverage)".format(ploidy, coverage),
             ha="center")
    fig.text(0.5, bottom_margin_frac-((bottom_margin_frac/2) + (bottom_margin_frac/4)), "Chromosome position [bases]",
             ha="center")
    fig.text(0.025, 0.5, "Coverage", rotation="vertical", va="center")
    fig.savefig(os.path.join(BASE_DIR, 'figures', assembly + '-cov.png'), dpi=256, transparent=True)